In [ ]:
import pandas as pd
import numpy as np
from google.colab  import drive
import os
drive.mount('/content/drive')

In [ ]:
# Get the file path input from the user
file_path = input("Enter the CSV filename (e.g., 'china_address.csv'): ").strip()

# Define the full path where your files are stored
default_file = "/content/drive/MyDrive/Business Intelligence/1. Control Techniques/12. Special Folder/Default_Category.csv"
input_file = f"/content/drive/MyDrive/Business Intelligence/1. Control Techniques/12. Special Folder/{file_path}"

# Check if the files exist
if not os.path.exists(default_file):
    raise FileNotFoundError(f"The default file '{default_file}' does not exist.")
if not os.path.exists(input_file):
    raise FileNotFoundError(f"The input file '{input_file}' does not exist.")

# Read the CSV files into DataFrames
default_df = pd.read_csv(default_file, encoding='latin1')
country_df = pd.read_csv(input_file, encoding='latin1')

# Validate column existence
if 'Category' not in default_df.columns or 'Category' not in country_df.columns:
    raise KeyError("The 'Category' column is missing in one or both files.")
if 'Business Model' not in default_df.columns or 'Business Model' not in country_df.columns:
    raise KeyError("The 'Business Model' column is missing in one or both files.")

# Ensure consistent formatting for default data (strip whitespace and lowercase)
country_df['Category'] = country_df['Category'].str.strip().str.lower().fillna('')
default_df['Category'] = default_df['Category'].str.strip().str.lower().fillna('')
country_df['Business Model'] = country_df['Business Model'].str.strip().str.lower().fillna('')
default_df['Business Model'] = default_df['Business Model'].str.strip().str.lower().fillna('')

# Extract default categories and valid pairs
default_categories = default_df['Category'].tolist()
valid_pairs = set(zip(default_df['Business Model'], default_df['Category']))

# Function to check and clean invalid categories
def clean_category(category, original_category):
    # Check if the original category is already valid (exact match)
    if original_category.lower() in [x.lower() for x in default_categories]:
        return original_category, False  # Don't change if it's already valid

    # Track if the category is valid or invalid
    for default in default_categories:
        if default in category.lower():
            return default.title(), False  # Return cleaned category and valid (False means valid)

    # Return original category and mark as invalid
    return original_category, True

# Apply the cleaning function and create the 'invalid_category' column
country_df['Category'], country_df['invalid_category'] = zip(*country_df['Category'].apply(lambda x: clean_category(x, x)))

# Function to validate the (Business Model, Category) combination
def validate_business_model(row):
    if (row['Business Model'], row['Category'].lower()) not in valid_pairs:
        return 1  # Mark as invalid
    return 0  # Mark as valid

# Update 'invalid_category' based on Business Model-Category combination
country_df['invalid_category'] = country_df.apply(validate_business_model, axis=1)

# Convert 'Category' and 'Business Model' columns to title case
country_df['Category'] = country_df['Category'].str.title()
country_df['Business Model'] = country_df['Business Model'].str.title()

# Define output path and create the directory if needed
output_dir = "/content/drive/MyDrive/Business Intelligence/1. Control Techniques/12. Special Folder/Output/"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, file_path)

# Save the updated DataFrame
country_df.to_csv(output_path, index=False)

print(f"File processed successfully. Saved to {output_path}")
